In [1]:
import auxiliary as aux

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


valid_true, valid = aux.train_valid_split(aux.data, frac=0.0125, seed=19)

## tests

In [7]:
%%time
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
    aux.Step(aux.transformer, 'all', imputer=SimpleImputer()),
).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 55/55 [00:00<00:00, 476.17it/s]


Final validation score: 0.8783267769946155
Overall final score: 0.10180127772818637
CPU times: user 1h 25min, sys: 2min 2s, total: 1h 27min 3s
Wall time: 13min 24s


In [9]:
%%time
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
    aux.Step(aux.cosine_stats, aux.subcol[1], threshold=0.7, subsample=0.05, backend='threading', seed=17),
    aux.Step(aux.transformer, 'all', imputer=SimpleImputer()),
).run(valid, validate_on=valid_true)

100%|██████████| 15/15 [12:57<00:00, 51.86s/it, avg. score=0.555]


Using `cosine_stats()` is deprecated! It may take a VERY long time on large data, work unstable or don't work at all.
87877 rows in 1758 chunks


100%|██████████| 1758/1758 [07:41<00:00,  3.81it/s]


Some values are NaN. Try decrease threshold.


Final validation: 100%|██████████| 55/55 [00:00<00:00, 461.12it/s]


Final validation score: 0.8806708728036292
Overall final score: 0.10207657602880504
CPU times: user 1h 40min 55s, sys: 10min 55s, total: 1h 51min 51s
Wall time: 20min 48s


In [ ]:
%%time
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
    aux.Step(aux.cosine_stats, aux.subcol[1], threshold=0.7, subsample=0.15, backend='threading', seed=17),
    aux.Step(aux.transformer, 'all', imputer=SimpleImputer()),
).run(valid, validate_on=valid_true)

In [2]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
#     aux.Step(aux.mice, aux.subcol[1] + aux.subcol[3], estimator=SGDRegressor(random_state=7, max_iter=1000), epochs=20, autosplit=False),
# ).run(valid, validate_on=valid_true)

## temp

In [4]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.cosine_stats, aux.subcol[1] + aux.subcol[2], threshold=0.7, backend='loky', chunksize=50, subsample=.0001),
# ).run(valid, validate_on=valid_true)

In [4]:
predicted.isna().any().any()

True

In [ ]:
aux.save_submission(predicted)

In [ ]:
#